## The Smart Cart Problem solving using Q-Learning

In [2]:
import gym 
#list the necessry library "gym" which contains the environment
env = gym.make("Taxi-v3").env
# env is the interface
#set the environmrnt using Taxi-V3
env.render()
#render it

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [3]:
env.reset() # Resets the environment and returns a random initial state.
env.render() # Step the environment by one timestep. Returns

print("The Action Space {}".format(env.action_space))
print("The State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

The Action Space Discrete(6)
The State Space Discrete(500)


The filled square represents the taxi, which is yellow without a passenger and green with a passenger.
he pipe ("|") represents a wall which the taxi cannot cross.
R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.
there is 6 possible Action including move east, move west, move south, move north, pick the passenger up and drop the passenger off.
As there is a 5*5 parking lot, 4 destinations and (4+1) passenger locations. Therefore, the total number of possible states will be 500.

In [12]:
state = env.encode(2, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 228
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


## The reward Table

In [14]:
env.P[228]
#{action: [(probability, nextstate, reward, done)]}

{0: [(1.0, 328, -1, False)],
 1: [(1.0, 128, -1, False)],
 2: [(1.0, 248, -1, False)],
 3: [(1.0, 208, -1, False)],
 4: [(1.0, 228, -10, False)],
 5: [(1.0, 228, -10, False)]}

All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)

## Solving the environment without Reinforcement Learning


In [15]:
env.s = 228

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("The Timesteps taken: {}".format(epochs))
print("The Penalties incurred: {}".format(penalties))

Timesteps taken: 722
Penalties incurred: 242


In [25]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(f"{frame['frame']}") 
        print(f"The Timestep: {i + 1}")
        print(f"The State: {frame['state']}")
        print(f"The Action: {frame['action']}")
        print(f"The Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

The Timestep: 722
The State: 0
The Action: 5
The Reward: 20


Our agent takes hundreds of timesteps and makes lots of wrong drop offs to deliver just one passenger to the right destination.

## Solving the environment with Reinforcement Learning

The whole process steps:
 - Initialize the Q-table by all zeros.
 - Start exploring actions: For each state, select any one among all possible actions for the current state (S).
 - Travel to the next state (S') as a result of that action (a).
 - For all possible actions from the state (S') select the one with the highest Q-value.
 - Update Q-table values using the equation.
 - Set the next state as the current state.
 - If goal state is reached, then end and repeat the process.


In [26]:
import numpy as np
#list the necessry library The "numpy" manage the data
q_table = np.zeros([env.observation_space.n, env.action_space.n])
#Initialize the Q-table by all zeros

The Q table (0,1,2,3,4,5)
- 0 represents South
- 1 represents North
- 2 represents East
- 3 represents West
- 4 represents pickup
- 5 represents dropoff

## Training the agent

In [39]:
import random
from IPython.display import clear_output

alpha = 0.1
gamma = 0.6
epsilon = 0.1
# Hyperparameters

all_epochs = []
all_penalties = []

for i in range(1, 150001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        Pre_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * Pre_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 150000
Training finished.



In [40]:
q_table[228]

array([ -2.3639511 ,  -2.3639511 ,  -2.3639511 ,  -2.1220864 ,
       -11.27325184, -11.27325184])

The maximum Q value is "west" (-2.1220864), so it looks like Q-Learning has effectively learned the best action to take in the illustration state!

## evaluating the agent

In [41]:
total_epochs, total_penalties = 0, 0
episodes = 500

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 500 episodes:
Average timesteps per episode: 13.168
Average penalties per episode: 0.0


After evaluating the agent, observed The agent's performance was significantly improved without any penalty, meaning it performed the correct pick-up and drop-off for 500 different passengers.